In [79]:
import json, re, sys, os
import numpy as np
import random as ra
from partitioner.tools import partitioner


In [81]:
import nltk

In [82]:
from nltk.tokenize import MWETokenizer
tokenizer = MWETokenizer()

In [83]:
def countPhrases(message):
    
    counts = {"*" : 0}
    
    for MWE in tokenizer.tokenize(message):
        phrase = "".join([block[1] for block in MWE])
        if re.search("[a-zA-Z]", phrase):
            counts.setdefault(phrase,0)
            counts[phrase] +=1
            counts["*"] +=1
    return counts

def loadPhrases(record):
    
    codes = record["codes"]
    counts = countPhrases(record["tweet"])
    
    for code in codes:
        codeCounts = counts
        if codes[code]:
            codeCounts["_"+code] += 1
            yield(code,codeCounts)
            
        else:
            codeCounts["_not." + code] =1
            yield("not." + code,codeCounts)
            
            
def aggregateByPhrase(d1,d2):
    
    for key in d2:
        d1.setdefault(key,0)
        d1[key] += d2[key]
        
    return d1

def findN(record):
    ID,counts = record
    yield("_ALLCODES",counts)
    counts["_N"] = float(len(counts.keys())-1)
    yield(ID,counts)
    
# in the following function, I don't know what is "N"  
def computeEntropies(record):
    code,entropies = record
    n = float(entropies["_N"])
    entropies["*"] = sum([entropies[phrase] for phrase in entropies if phrase != "_N" and phrase!="*" and phrase!="_"+code])
    
    m = float(entropies["*"])
    del entropies["_N"]
    del entropies["*"]
    alpha = n/N
    
    for phrase in entropies:
        if not re.match("^_",phrase):
            entropy = -np.log2((entropies[phrase]+alpha)/(alpha*N+m))
            entropies[phrase] = entropy
            
    entropies["_DEFAULT"] = -np.log2(alpha/(alpha*N + m))
    
    return code,entropies


def processTweet(tweet):
    result = {
        "posteriors" : {code: -prior_entropy[code] for code in prior_entropy},
        "M_words" : 0,
        "counts" : {},
        "N" : 0,
        "M" : 0,
        "tweet" : tweet
    }
    
    review_M = 0
    tempCounts = countPhrasest(tweet)
    for phrase in tempCounts:
        if phrase!= "*":
            result['counts'].setdefault(phrase,0)
            result['counts'][phrase] += tempCounts[phrase]
            
    todel = []
    
    for phrase in result['counts']:
        if phrase!="*":
            INLIKE = False
            for code in likelihoodEntropy:
                if likelihoodEntropy[code].get(phrase,False):
                    result['posteriors'][code] -= result['counts'][phrase] * likelihoodEntropy[code][phrase]
                    INLIKE = True
                    
                if INLIKE:
                    result['M'] += result['counts'][phrase]
                    result["M_words"] += result['counts'][phrase] * len(re.split(" ",phrase))
                    result["N"] += 1
                    
                    for code in likelihoodEntropy:
                        if not likelihoodEntropy[code].get(phrase,False):
                            result['posteriors'][code] -= result['counts'][phrase]*likelihoodEntropy[code]["_DEFAULT"]
                    
                    result['counts'][phrase] = int(result['counts'][phrase])
                    
                else:
                    todel.append(phrase)
                    
    for phrase in todel:
        del result['counts'][phrase]
        
    newPosteriors = {}
    
    for code in result['posteriors']:
        code = re.split("\.",code)
        if len(code) ==2:
            code = code[1]
            newPosteriors.setdefault(code,{})
            newPosteriors[code]["0"] = result["posteriors"][code]
            
        else:
            code = code
            newPosteriors.setdefault(code,{})
            newPosteriors[code]["1"] = result['posteriors'][code]
            
    result["posteriors"] = newPosteriors
    
    
    for code in result['posteriors']:
        
        if len(result['posteriors'][code].keys()) == 2:
            P0 = 1/(1 + (2**( result['posteriors'][code]["1"]-result['posteriors'][code]["0"])))
            
            P1 = 1 - P0
            
            result['posteriors'][code] = {"0": P0, "1":P1}
            
        else:
            result['posteriors'][code] = {"0":1, "1":0}
            
    result["M"] = int(result["M"])
    result["M_words"] = int(result["M_words"])
    
    
    return result

#### Training the Model

In [84]:
# converting the given csv file to json file to correspond with the given code:

import csv
import json


# Function to convert a CSV to JSON
# Takes the file paths as arguments
def make_json(csvFilePath, jsonFilePath):
	
	# create a dictionary
	data = {}
	
	# Open a csv reader called DictReader
	with open(csvFilePath, encoding='utf-8') as csvf:
		csvReader = csv.DictReader(csvf)
		
		# Convert each row into a dictionary
		# and add it to data
		for ind,rows in enumerate(csvReader):
			
			# Assuming a column named 'No' to
			# be the primary key
			key = ind
			data[key] = dict()
			data[key]["codes"] = rows
			data[key]["tweet"] = rows["tweet"]
			del data[key]["codes"]["tweet"]


	# Open a json writer, and use the json.dumps()
	# function to dump data
	with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
		jsonf.write(json.dumps(data, indent=4))
		
# Driver Code

# Decide the two file paths according to your
# computer system
csvFilePath = r'datasets/dataverse_files/coded-tweets-training.csv'
jsonFilePath = r'datasets/dataverse_files/trainingData.json'

# Call the make_json function
make_json(csvFilePath, jsonFilePath)

In [85]:
# trainingData.json created

In [89]:
with open(r'datasets/dataverse_files/trainingData.json',encoding='utf-8') as m:
    for row in m:
        record = json.loads(row)
        print(record)
        print("ruchika")

JSONDecodeError: Expecting property name enclosed in double quotes: line 2 column 1 (char 2)

In [ ]:
with open("datasets/dataverse_files/trainingData.json",'r',encoding='utf-8') as filey:
    for line in filey:
        
    

In [87]:
codes = [
    "collective_force",
    "collective_peace",
    "singular_force",
    "singular_peace",
    "collective",
    "singular",
    "peace",
    "force",
    "action"
]

thresholds = [(x)/100 for x in range(1,100)]

records = []

with open("datasets/dataverse_files/trainingData.json",'r') as file:
    
    c = 0
    for line in file:
        record = json.loads(line)
        oldcodes = dict(record[c]["codes"])
        for code in oldcodes:
            for newcode in re.split("_",code):
                if not record[c]["codes"].get(newcode,0):
                    record[c]["codes"][newcode] = oldcode[code]
                
                if not record[c]["codes"].get("action",0):
                    record[c]["codes"]["action"] = oldcodes[code]
                    
            records.append(record)
        c+=1
        
ra.seed(0)

ra.shuffle(records)

numfolds = 10
folds = []

foldsize = int((len(records)/numfolds) + 0.5 )
if foldsize * numfolds < len(records):
    foldsize += 1

allresults = []

for foldnum in range(numfolds):
    
    trainingrecords = []
    testingrecords = []
    
    for k in range(numfolds):
        
        if k==foldnum:
            testingrecords.extend(list(records[k*foldsize : (k+1)*foldsize]))
            
        else:
            trainingrecords.extend(listt(records[k*foldsize : (k+1)*foldsize]))
            
            
    ###### training ######
    ######################
    
    allcounts = {}
    for code in codes:
        allcounts[code] = {}
        allcounts["not."+code] = {}
        
        
    total_tweets = 0
    
    for record in trainingrecords:
        total_tweets += 1
        for k,v in loadPhrases(record):
            allcounts[k] = aggregateByPhrase(allcounts[k],v)
            
    for code in codes:
        allcounts['_ALLCODES'] = aggregateByPhrase(allcounts[code], allcounts["not."+code])
        break
        
    for ky in allcounts.keys():
        allcounts[ky]["_N"] = float(len(allcounts[ky].keys()) -1)
        
    N = float(len(allcounts["_ALLCODES"].keys())) -2
    
    
    del allcounts["_ALLCODES"]
    
    entropies = list()
    
    for ky in allcounts.keys():
        
        entropies.append(computeEntropies((ky, allcounts[ky])))
        
    likelihoodEntropy = {}
    prior_entropy = {}
    
    for x in entropies:
        
        code = x[0]
        entdict = x[1]
        priorNum = entdict["_"+code]
        prior_entropy[code] = priorNum
        del entdict["_"+code]
        likelihoodEntropy[code] = entdict
        
        
        
    for code in prior_entropy:
        prior_entropy[code] = -np.log2(prior_entropy[code]/total_tweets)
        
        
        
    ###############end of training###############
    #############################################
    
    
    ###############testing testing testing###############
    #####################################################
    
    
    totals = {}
    for code in codes:
        totals[code] = {}
        for thresh in thresholds:
            totals[code][str(thresh)] = {
                "TP": 0,
                "FP":0,
                "FN":0,
                "TN":0
            }
        
    for record in testingrecords:
        result = processTweet(record["tweet"])
        
        for code in codes:
            truth = records["codes"][code]
            for thresh in thresholds:
                
                st = str(thresh)
                
                if result["posteriors"][code]["1"] >= thresh:
                    prediction = 1
                else:
                    prediction = 0
                    
                if truth:
                    if prediction:
                        totals[code][st]["TP"] +=1
                        
                    else:
                        totals[code][st]["FN"] +=1
                        
                else:
                    if prediction:
                        totals[code][st]["FP"] +=1
                        
                        
                    else:
                        totals[code][st]["TN"] +=1
                        
                        
        for code in codes:
            for thresh in thresholds:
                st = str(thresh)
                
                try:
                    totals[code][st]["TPR"] = totals[code][st]["TP"]/(totals[code][st]["TP"] + totals[code][st]["FN"])
                except:
                    totals[code][st]["TPR"] = 0
                    
                    
                try:
                    totals[code][st]["PPV"] = totals[code][st]["TP"]/(totals[code][st]["TP"]+totals[code][st]["FP"])
                
                except:
                    totals[code][st]["PPV"] - 0
                    
                    
                try:
                    totals[code][st]["F1"] = (2*totals[code][st]["TPR"]*totals[code][st]["PPV"])/(
                    totals[code][st]["TPR"] + totals[code][st]["PPV"])
                    
                except:
                    totals[code][st]["F1"] = 0
                    
                try:
                    totals[code][st]["ACC"] = (2*totals[code][st]["TP"]*totals[code][st]["TN"])/(
                    totals[code][st]["TP"] + totals[code][st]["TN"]) + totals[code][st]["FP"]+ totals[code][st]["FN"]
                    
                except:
                    
                    totals[code][st]["ACC"] = 0
                    
                    
        print("the results for fold: "+str(foldnum))
        
        for code in codes:
            st = max([(totals[code][str(thresh)]["F1"], str(thresh)) for thresh in thresholds], key = lambda x: x[0])[1]
            
            totals[code]["thresh"] = st
            
            print(code,st,totals[code][st]["PPV"], totals[code][st]["TPR"], totals[code][st]["F1"], totals[code][st]["ACC"])
            
            print(allresults.append(totals))
            
            ##################################################
            ### F1 optimal posterior ###
bestThresh = {}
for code in codes:
    bestThresh[code] = [0, 0]
    for thresh in thresholds:
        F1 = np.mean([totals[code][str(thresh)]["F1"] for totals in allcounts])
        
        if F1 > bestThresh[code][1]:
            bestThresh[code] = [thresh,F1]
            
        thresh = bestThresh[code][0]
        
        print(code,thresh,np.mean([totals[code][str(thresh)]["PPV"] for totals in allresults]),
             np.mean([totals[code][str(thresh)]["TPR"] for totals in allresults]),
             np.mean([totals[code][str(thresh)]["F1"] for totals in allresults]))
        

JSONDecodeError: Expecting property name enclosed in double quotes: line 2 column 1 (char 2)

In [34]:
record = {"codes":{'collective-force':0,"singular-foce":0,"collective-peace":0,"singular-peace":1}}

In [35]:
oldcodes = dict(a["codes"])

In [36]:
oldcodes

{'collective-force': 0,
 'singular-foce': 0,
 'collective-peace': 0,
 'singular-peace': 1}

In [38]:
records = list()
for code in oldcodes:
    for newcode in re.split("-",code):
        if not record["codes"].get(newcode,0):
            record["codes"][newcode] = oldcodes[code]
        if not record["codes"].get("action",0):
            record["codes"]["action"] = oldcodes[code]
records.append(record)
            

In [39]:
a

{'codes': {'collective-force': 0,
  'singular-foce': 0,
  'collective-peace': 0,
  'singular-peace': 1,
  'collective': 0,
  'action': 1,
  'force': 0,
  'singular': 1,
  'foce': 0,
  'peace': 1}}

In [73]:
import json, csv

with open(r'datasets/dataverse_files/coded-tweets-training.csv',encoding='utf-8') as f:
    csvf = csv.DictReader(f)
    for ind,row in enumerate(csvf):
        del row['tweet']
        print(ind,row)

0 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
1 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
2 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
3 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
4 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
5 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
6 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
7 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
8 {'singular_peace': '1', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
9 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
10 {'singu

1431 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
1432 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
1433 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
1434 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
1435 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
1436 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
1437 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
1438 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
1439 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
1440 {'singular_peace': '1', 'singular_force': '0', 'collective_peace': '0', 'coll

2930 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
2931 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
2932 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
2933 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
2934 {'singular_peace': '1', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '1'}
2935 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
2936 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
2937 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
2938 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
2939 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'coll

4430 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
4431 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
4432 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
4433 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
4434 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
4435 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
4436 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
4437 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
4438 {'singular_peace': '1', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
4439 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'coll

5930 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
5931 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
5932 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
5933 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
5934 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
5935 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
5936 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
5937 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
5938 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
5939 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'coll

...

14429 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
14430 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
14431 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
14432 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
14433 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
14434 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
14435 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
14436 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
14437 {'singular_peace': '1', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
14438 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': 

16178 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
16179 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
16180 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
16181 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
16182 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
16183 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
16184 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
16185 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
16186 {'singular_peace': '1', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '1'}
16187 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': 

17792 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
17793 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
17794 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
17795 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
17796 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
17797 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
17798 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
17799 {'singular_peace': '1', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
17800 {'singular_peace': '1', 'singular_force': '0', 'collective_peace': '1', 'collective_force': '1'}
17801 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': 

19428 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
19429 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
19430 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
19431 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
19432 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
19433 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
19434 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
19435 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
19436 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
19437 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': 

21178 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
21179 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
21180 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
21181 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
21182 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
21183 {'singular_peace': '1', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
21184 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
21185 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
21186 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': '0', 'collective_force': '0'}
21187 {'singular_peace': '0', 'singular_force': '0', 'collective_peace': 